# Пробуем простую полносвязную сеть

## Загрузка данных

In [22]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
plt.style.use('seaborn')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
from pathlib import Path
from skimage import transform
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.preprocessing import OrdinalEncoder
from read_data import *
from read_data import good_cols
%matplotlib inline

In [ ]:
all_df = get_df()
TARGET_LEN = 2000
activities = sep_by_len(all_df, 20000)
cut_df = cut_act(activities, TARGET_LEN, count=4)
n_df = normalize_df(cut_df)
n_df.sample()

 11%|█████████▎                                                                          | 1/9 [00:03<00:30,  3.81s/it]

Получаем вектор признаков

In [ ]:
X = n_df.iloc[:, 1:].values

Y = n_df.iloc[:, 0] # целевая переменная
X.shape, Y.shape

In [ ]:
X_flat = get_flatten(X)
X_flat.shape

In [ ]:
def y_encode(y_data):
    y_targ = np.zeros_like(y_data)
    code = list(np.unique(y_data))
    for i, el in enumerate(y_data):
        y_targ[i] = code.index(el)
        
    return np.array(y_targ, dtype = np.int32)

In [ ]:
y_targ = y_encode(Y)
y_targ

In [ ]:
X_train_tensor, X_val_tensor, y_train_tensor, y_val_tensor = \
    train_test_split(torch.FloatTensor(X_flat), torch.LongTensor(y_targ), random_state=42, test_size = 0.3)
X_train_tensor.shape, y_train_tensor.shape, np.unique(y_train_tensor), np.unique(y_val_tensor)

In [ ]:
class SignalDataset(Dataset):
    def __init__(self, data_X, data_Y):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.X = data_X
        self.Y = data_Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        input_data = self.X[idx]
        label = self.Y[idx]
        
        return input_data, label

In [ ]:
BATCH_SIZE = 64

trainset = SignalDataset(X_train_tensor, y_train_tensor)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=0)

testset = SignalDataset(X_val_tensor, y_val_tensor)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=0)
len(trainset)

## Подготовка модели

In [ ]:
activation = nn.LeakyReLU

net = torch.nn.Sequential(
    nn.Linear(31*TARGET_LEN, 128),
    activation(),
    nn.Linear(128,128),
    activation(),
    nn.Linear(128,len(np.unique(y_targ)))
)

## Обучение

In [63]:
NUM_EPOCHS = 100

loss_fn = torch.nn.CrossEntropyLoss(size_average=False)
losses = []

learning_rate = 3e-3
optimizer = torch.optim.AdamW(net.parameters(), lr=learning_rate)
ebar = tqdm(range(NUM_EPOCHS))

for epoch_num in ebar:
    running_loss = 0.0
    iter_num = 0
#     pbar = tqdm(enumerate(trainloader), total = len(trainloader))
    pbar = enumerate(trainloader)
    for iter_num, (X_batch, y_batch) in pbar:
        iter_num +=1
        # forward (подсчёт ответа с текущими весами)
        y_pred = net(X_batch)
        
        # вычисляем loss'ы
        loss = loss_fn(y_pred, y_batch)
        running_loss += loss.item()
                     
        # зануляем градиенты
        optimizer.zero_grad()

        # backward (подсчёт новых градиентов)
        loss.backward()

        # обновляем веса
        optimizer.step()
        
    line = '[{}/{}] current loss: {}'.format(epoch_num, NUM_EPOCHS, running_loss/len(trainset))
    ebar.set_description(line)
    
    losses.append(running_loss/len(trainset))
        
    with torch.no_grad():
        y_pred = torch.softmax(net(X_val_tensor), 1)
        y = torch.argmax(y_pred, axis=1)
        line = '[{}/{}] current valid score: {}'.format(epoch_num+1, NUM_EPOCHS, f1_score(y_val_tensor, y, average='macro'))
        print(line)
#         ebar.set_description(line)

C:\Users\kitro\anaconda3\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
[0/100] current loss: 25.463682469313706:   1%|▍                                       | 1/100 [00:00<01:02,  1.58it/s]

[1/100] current valid score: 0.02040816326530612


[1/100] current loss: 11.788713718817487:   2%|▊                                       | 2/100 [00:01<00:59,  1.65it/s]

[2/100] current valid score: 0.011494252873563218


[2/100] current loss: 4.663786477189723:   3%|█▏                                       | 3/100 [00:01<00:56,  1.71it/s]

...

[98/100] current valid score: 0.3953042328042328


[98/100] current loss: 0.000488453730183646:  99%|███████████████████████████████████▋| 99/100 [00:57<00:00,  1.58it/s]

[99/100] current valid score: 0.3953042328042328


[99/100] current loss: 0.0004893214123823294: 100%|██████████████████████████████████| 100/100 [00:58<00:00,  1.71it/s]

[100/100] current valid score: 0.3953042328042328


In [ ]:
fig, axs = plt.subplots(1,1,figsize=(20,8))
plt.plot(np.arange(NUM_EPOCHS), losses)
plt.show()

In [ ]:
for i in range(len(np.unique(y_val_tensor))):
    idx = np.argwhere(y_val_tensor==i)[0]
    y_pred = torch.sigmoid(net(X_val_tensor[idx]))
#     print(torch.sum(y_pred,axis=1))
#     print(X_val_tensor.shape, idx.shape, y_val_tensor.shape)
    y = torch.argmax(y_pred, axis=1)
#     print(y_pred)
    print(np.unique(Y)[i], len(y[y==i])/len(idx))

In [266]:
with torch.no_grad():
    y_pred = torch.softmax(net(X_val_tensor), 1)
    y = torch.argmax(y_pred, axis=1)
#     print(net(X_val_tensor).shape)
    print(roc_auc_score(y_val_tensor, y_pred, multi_class='ovo'))
    print(f1_score(y_val_tensor, y, average='macro'))

0.9093150715513585
0.7409394506625
